In [136]:
!pip install python-Levenshtein
!pip install py_stringmatching
!pip install pyjarowinkler
!pip install owlready2
!pip install nltk
!pip install strsimpy
!pip install scikit-learn scipy matplotlib


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [137]:
from py_stringmatching import utils
from py_stringmatching.similarity_measure.jaro_winkler import JaroWinkler
from py_stringmatching.similarity_measure.hybrid_similarity_measure import \
                                                    HybridSimilarityMeasure


class MongeElkan(HybridSimilarityMeasure):
    """Computes Monge-Elkan measure.

    The Monge-Elkan similarity measure is a type of hybrid similarity measure that combines the benefits of
    sequence-based and set-based methods. This can be effective for domains in which more control is needed
    over the similarity measure. It implicitly uses a secondary similarity measure, such as Levenshtein to compute
    over all similarity score. See the string matching chapter in the DI book (Principles of Data Integration). 

    Args:
        sim_func (function): Secondary similarity function. This is expected to be a sequence-based
                             similarity measure (defaults to Jaro-Winkler similarity measure).

    Attributes:
        sim_func (function): An attribute to store the secondary similarity function.
    """

    def __init__(self, sim_func=JaroWinkler().get_raw_score):
        self.sim_func = sim_func
        super(MongeElkan, self).__init__()

    def get_raw_score(self, bag1, bag2):
            """Computes the raw Monge-Elkan score between two bags (lists).

            Args:
                bag1,bag2 (list): Input lists.

            Returns:
                Monge-Elkan similarity score (float).

            Raises:
                TypeError : If the inputs are not lists or if one of the inputs is None.

            Examples:
                >>> me = MongeElkan()
                >>> me.get_raw_score(['Niall'], ['Neal'])
                0.8049999999999999
                >>> me.get_raw_score(['Niall'], ['Nigel'])
                0.7866666666666667
                >>> me.get_raw_score(['Comput.', 'Sci.', 'and', 'Eng.', 'Dept.,', 'University', 'of', 'California,', 'San', 'Diego'], ['Department', 'of', 'Computer', 'Science,', 'Univ.', 'Calif.,', 'San', 'Diego'])
                0.8677218614718616
                >>> me.get_raw_score([''], ['a'])
                0.0
                >>> me = MongeElkan(sim_func=NeedlemanWunsch().get_raw_score)
                >>> me.get_raw_score(['Comput.', 'Sci.', 'and', 'Eng.', 'Dept.,', 'University', 'of', 'California,', 'San', 'Diego'], ['Department', 'of', 'Computer', 'Science,', 'Univ.', 'Calif.,', 'San', 'Diego'])
                2.0
                >>> me = MongeElkan(sim_func=Affine().get_raw_score)
                >>> me.get_raw_score(['Comput.', 'Sci.', 'and', 'Eng.', 'Dept.,', 'University', 'of', 'California,', 'San', 'Diego'], ['Department', 'of', 'Computer', 'Science,', 'Univ.', 'Calif.,', 'San', 'Diego'])
                2.25

            References:
                * Principles of Data Integration book
            """
            
            # input validations
            utils.sim_check_for_none(bag1, bag2)
            utils.sim_check_for_list_or_set_inputs(bag1, bag2)

            # if exact match return 1.0
            if utils.sim_check_for_exact_match(bag1, bag2):
                return 1.0

            # if one of the strings is empty return 0
            if utils.sim_check_for_empty(bag1, bag2):
                return 0

            # aggregated sum of all the max sim score of all the elements in bag1
            # with elements in bag2
            sum_of_maxes = 0
            for el1 in bag1:
                max_sim = float('-inf')
                for el2 in bag2:
                    max_sim = max(max_sim, self.sim_func(el1, el2))
                sum_of_maxes += max_sim

            sim = float(sum_of_maxes) / float(len(bag1))

            return sim


    def get_sim_func(self):
            """Get the secondary similarity function.

            Returns:
                secondary similarity function (function).
            """
            return self.sim_func
    def set_sim_func(self, sim_func):
            """Set the secondary similarity function.

            Args:
                sim_func (function): Secondary similarity function.
            """
            self.sim_func = sim_func
            return True
me = MongeElkan()
me.get_raw_score(['Niall'], ['Neal'])
# me.get_raw_score(['Niall'], ['Nigel'])

0.8050000071525574

N gram Similarity

In [138]:
def ngram(sentence, num):
    tmp = [] 
    sent_len = len(sentence) - num +1
    for i in range(sent_len):
        tmp.append(sentence[i:i+num]) 
    return tmp

def diff_ngram(sent_a, sent_b, num):
    a = ngram(sent_a, num)
    b = ngram(sent_b, num) 
    common = [] 
    cnt = 0 
    for i in a:
        for j in b:
            if i == j:
                cnt += 1
                common.append(i)
    return cnt/len(a), common

c = "Niall"
d = "Nigel"

r2, word2 = diff_ngram(c, d ,1)
print("2-gram: ", r2, word2)

2-gram:  0.8 ['N', 'i', 'l', 'l']


Cosine Simililarity

In [139]:
import re
import math
from collections import Counter


def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    word = re.compile(r'\w+')
    words = word.findall(text)
    return Counter(words)


def get_result(content_a, content_b):
    text1 = content_a
    text2 = content_b

    vector1 = text_to_vector(text1)
    vector2 = text_to_vector(text2)

    cosine_result = get_cosine(vector1, vector2)
    return cosine_result

print(get_result('restaurant', 'restaurant')) #0.65565


1.0


Levenshtein Similairity

In [140]:
import Levenshtein
Levenshtein.distance("Niall", "Nigel")

2

Aspect in Ontology

In [141]:
['AMBIENCE#GENERAL']
['DRINKS#PRICES', 'DRINKS#QUALITY', 'DRINKS#STYLE_OPTIONS']
['RESTAURANT#MISCELLANEOUS']
['LOCATION#GENERAL']
['DRINKS#PRICES', 'FOOD#PRICES', 'RESTAURANT#PRICES']
['RESTAURANT#GENERAL']
['SERVICE#GENERAL']
['DRINKS#STYLE_OPTIONS', 'FOOD#STYLE_OPTIONS']
['FOOD#PRICES', 'FOOD#QUALITY', 'FOOD#STYLE_OPTIONS']

#Catatan
#sustenance enggak tau masuk aspek yang mana

['FOOD#PRICES', 'FOOD#QUALITY', 'FOOD#STYLE_OPTIONS']

Read XML

In [142]:
def ReadXml(dom,topic):
    xml_text = []
    xml_target = []
    xml_aspect = []
    xml_sentiment = []
    for node in topic:
        opini=node.getElementsByTagName('Opinion')
        temp_target = []
        temp_aspect = []
        temp_sentiment = []
        for o in opini:
            temp_target.append(o.attributes['target'].value)
            temp_aspect.append(o.attributes['category'].value)
            temp_sentiment.append(o.attributes['polarity'].value)
        xml_text.append(node.getElementsByTagName('text')[0].firstChild.data)
        xml_target.append(temp_target)
        xml_aspect.append(temp_aspect)
        xml_sentiment.append(temp_sentiment)
    return xml_text, xml_target, xml_aspect, xml_sentiment

Split Text

In [143]:
def split_delimiter(text):
    text_split = text.split(", and ")
    text_split_arr = []
    text_split_result_arr = []
    for text1 in text_split:
    #     print(text)
        text_split = text1.split(" and ")
        for text_split_result in text_split:
            text_split_arr.append(text_split_result)
    for text1 in text_split_arr:
        text_split = text1.split(", ")
        for text_split_result in text_split:
            text_split_result_arr.append(text_split_result)
    return text_split_result_arr

import re
def split_delimiter2(text):
    return re.split('; |, |, and | and ', text)

text = "Ive asked a cart, attendant for and lotus leaf wrapped rice, and she replied back rice and just; walked; away"
text_split = split_delimiter2(text)
print(text)
print(text_split)

Ive asked a cart, attendant for and lotus leaf wrapped rice, and she replied back rice and just; walked; away
['Ive asked a cart', 'attendant for', 'lotus leaf wrapped rice', 'and she replied back rice', 'just', 'walked', 'away']


Split Data

In [144]:
def jml_data_train_persentase(xml_text,persentase_train):
    jml_train = round(len(xml_text)*persentase_train)
    return jml_train

def split_to_test_and_train(xml_text, xml_target, xml_aspect, xml_sentiment,jml_train):
    xml_text_train = xml_text[0:jml_train]
    xml_text_test = xml_text[jml_train:len(xml_text)]
    xml_target_train = xml_target[0:jml_train]
    xml_target_test = xml_target[jml_train:len(xml_text)]
    xml_aspect_train = xml_aspect[0:jml_train]
    xml_aspect_test = xml_aspect[jml_train:len(xml_text)]
    xml_sentiment_train = xml_sentiment[0:jml_train]
    xml_sentiment_test = xml_sentiment[jml_train:len(xml_text)]
    return xml_text_train, xml_text_test, xml_target_train, xml_target_test, xml_aspect_train, xml_aspect_test, xml_sentiment_train, xml_sentiment_test

Preprocessing

In [145]:
def preprocessing(sentence):
    # gunakan fungsi .lower() untuk case folding
    lowercase_sentence = sentence.lower()

    #remove angka
    lowercase_sentence = re.sub(r"\d+", "", lowercase_sentence)

    #remove whitespace leading & trailing
    lowercase_sentence = lowercase_sentence.strip()

    #remove multiple whitespace into single whitespace
    lowercase_sentence = re.sub('\s+',' ',lowercase_sentence)

    tokens = nltk.tokenize.word_tokenize(lowercase_sentence)
#     print(tokens)
# #     #Lemmatization
#     lemma = []
#     wordnet_lemmatizer = WordNetLemmatizer()
#     for w in tokens:
#     #     print("Lemma for {} is {}".format(w, wordnet_lemmatizer.lemmatize(w)))
#         lemma.append(wordnet_lemmatizer.lemmatize(w))
        
    # Stemming    
#     from nltk.stem.porter import PorterStemmer
#     porter_stemmer  = PorterStemmer()
#     for w in tokens:
# #         print("Stemming for {} is {}".format(w,porter_stemmer.stem(w)))
#         lemma.append(porter_stemmer.stem(w))
    return tokens

Print multi aspect

In [146]:
def print_data(xml_text, xml_target, xml_aspect, xml_sentiment,jml_print):
    row = "{name1:^20}|{name2:^20}|{name3:^20}".format
#     for index_text in range(len(xml_text)):
    for index_text in range(jml_print):
        print("Text: "+str(xml_text[index_text]))
        for i in range(len(xml_target[index_text])):
            print("- Target: "+str(xml_target[index_text][i])+" | Aspect: "+str(xml_aspect[index_text][i])+" | Sentiment: "+str(xml_sentiment[index_text][i]))
        print("")

Print One aspect

In [147]:
def print_data_split_text(xml_text, xml_target, xml_aspect, xml_sentiment,jml_print):
    row = "{name1:^20}|{name2:^20}|{name3:^20}".format
#     for index_text in range(len(xml_text)):
    for index_text in range(jml_print):
        print("Text: "+str(xml_text[index_text]))
        print("- Target: "+str(xml_target[index_text])+" | Aspect: "+str(xml_aspect[index_text])+" | Sentiment: "+str(xml_sentiment[index_text]))
        print("")

Delete multi index

In [148]:
def deleted_multi_index(arr,indexes):
    for index in sorted(indexes, reverse=True):
        del arr[index]
    return arr

arr = [0,1,2,3,4,5,6,7,8,9,10]
indexes = [2, 3, 5]
deleted_multi_index(arr,indexes)

[0, 1, 4, 6, 7, 8, 9, 10]

Double target

In [149]:
def search_index_multi(arr,value):
    return [i for i, x in enumerate(arr) if x == value]

def clear_double_target(xml_text, xml_target, xml_aspect, xml_sentiment):
    for index_text in range(len(xml_text)):
        if (len(xml_target[index_text])>1):
#             print("1"+str(xml_target[index_text]))
            jml_xml_target = len(xml_target[index_text]) - 1
#             for i in range(jml_xml_target):
            i = 0    
            while i < jml_xml_target :
#                 print(search_index_multi(xml_target[index_text],xml_target[index_text][i]))
                arr_pop = search_index_multi(xml_target[index_text],xml_target[index_text][i])
                i +=1
                index_pop = len(arr_pop) -1
                while index_pop>0:
#                     print(xml_target[index_text][arr_pop[index_pop]])
#                     print("-- "+str(arr_pop[index_pop]))
                    xml_target[index_text].pop(arr_pop[index_pop])
                    xml_aspect[index_text].pop(arr_pop[index_pop])
                    xml_sentiment[index_text].pop(arr_pop[index_pop])
                    index_pop -= 1
            
                jml_xml_target = len(xml_target[index_text]) - 1
#                 xml_target[index_text].pop(0)
#                 jml_xml_target = jml_xml_target - 1
#             print("2"+str(xml_target[index_text]))
    return xml_text, xml_target, xml_aspect, xml_sentiment

x_text = ["Service was devine, oysters where a sensual as they come, and the price can't be beat!!!"]
x_target = [["restaurant","restaurant"]]
x_aspect = [["RESTAURANT#GENERAL","FOOD#QUALITY"]]
x_sentiment = [["positif","positif"]]
print("Data awal")
print_data(x_text, x_target, x_aspect, x_sentiment,1)

x_text, x_target, x_aspect, x_sentiment = clear_double_target(x_text, x_target, x_aspect, x_sentiment)
print("Hasil")
print_data(x_text, x_target, x_aspect, x_sentiment,1)

Data awal
Text: Service was devine, oysters where a sensual as they come, and the price can't be beat!!!
- Target: restaurant | Aspect: RESTAURANT#GENERAL | Sentiment: positif
- Target: restaurant | Aspect: FOOD#QUALITY | Sentiment: positif

Hasil
Text: Service was devine, oysters where a sensual as they come, and the price can't be beat!!!
- Target: restaurant | Aspect: RESTAURANT#GENERAL | Sentiment: positif



Search Index Value yg sama

In [150]:
add = [1,2,3,4,5,6,6,7,8,9,7,6,4,6]
print(add.index(6))

def index_multi(arr,value):
    return [i for i, x in enumerate(arr) if x == value]

a = index_multi(add,6)
print(a)

5
[5, 6, 11, 13]


Preprocessing multi aspect to one aspect

In [151]:
import xml.dom.minidom
from xml.dom.minidom import Node
import nltk
import re
from nltk.stem import 	WordNetLemmatizer
from owlready2 import *

from nltk import pos_tag
from nltk import RegexpParser
nltk.download('all')
# def chunking22(text):
#     result = []
#     result_chunking = []
#     tokens_tag = pos_tag(text)
#     patterns= "NP:{<DT|PP|CD|RB>?<JJ|JJR|JJS>*<NN|NNS|PRP|NNP|VB|IN|PRP\$>+<VBD|VBZ|VBN|VBP|VB|IN>*<JJ|JJS|RB>*<PRP|NN|NNS>*}"
#     chunker = RegexpParser(patterns)
# #     print("After Regex:",chunker)
#     output = chunker.parse(tokens_tag)
#     print("After Chunking",output)
# #     print("")
# #     print("After Chunking Parse")
# #     for i in output:
# # #         print(i)
# # #         print(len(i))
# #         result_chunking = []
# #         for j in i:
# # #             print(j[0])
# #             result_chunking.append(j[0])
# #         result.append(result_chunking)
# #     return result  

def ontology_ABSA_mention(sentence_1):
    result_mention = []
    result_sentiment = []
    result_mention_aspect = []
    result_mention_subclass = []
#     print ("{:<14} {:<13} {:<15} {:<10} {:<15}".format("Term", "Sentiment","Aspect", "Sub_Class", "lexicon"))
#     print("----------------------------------------------------------------")
#     print("total mention"+str(mention))
#     print("total sentence"+str(sentence_1))
    for x in range(len(sentence_1)):
        for i in range(len(mention)):
            if sentence_1[x] == mention[i]:
                result_mention.append(mention[i])
                result_sentiment.append(mention_sentiment[i])
                result_mention_aspect.append(mention_aspect[i])
                result_mention_subclass.append(mention_subclass[i])
                
#     for i in range(len(mention)):
# #         print("mention  "+mention[i])
#         if mention[i] in sentence_1:
# #             print ("{:<14} {:<13} {:<15} {:<10} {:<15}".format(str(mention[i]), str(mention_sentiment[i]),str(mention_aspect[i]), str(mention_subclass[i]),str(mention_lex[i])))
#             result_mention.append(mention[i])
#             result_sentiment.append(mention_sentiment[i])
#             result_mention_aspect.append(mention_aspect[i])
#             result_mention_subclass.append(mention_subclass[i])
    
    return result_mention,result_sentiment,result_mention_aspect,result_mention_subclass

def ontology_ABSA_sentiment(sentence_1):
    result_sentiment_term = []
    result_sentiment = []
    result_sentiment_aspect = []
#     print ("{:<14} {:<13} {:<15} ".format("Term", "Sentiment","Aspect"))
#     print("----------------------------------------------------------------")
    for i in range(len(sentiment_term)-1):
        if sentiment_term[i] in sentence_1:
#             print("{:<14} {:<13} {:<15}".format(str(sentiment_term[i]),str(sentiment[i]),str(sentiment_aspect[i])))
            result_sentiment_term.append(sentiment_term[i])
            result_sentiment.append(sentiment[i])
            result_sentiment_aspect.append(sentiment_aspect[i])
            
    return result_sentiment_term,result_sentiment,result_sentiment_aspect

## Get Term from Ontology
# onto = get_ontology("C:/Users/akhmad.saifullah/opencv/Scripts/Tesis/ontology.owl").load()

# ## Get Data
# dom = xml.dom.minidom.parse("D:/Paper S2/Data/Semval-2016/ABSA16_Restaurants_Train_SB1_v2 (2000).xml")
# # dom = xml.dom.minidom.parse("D:/Paper S2/[Matakuliah] Implementasi Proyek 2/SemEval2016 6000 data/EN_REST_SB2_TEST.xml.gold")
# text=dom.getElementsByTagName('text')
# print(len(text))

dom = xml.dom.minidom.parse("ABSA16_Restaurants_Train_SB1_v2 (2000).xml")
# dom = xml.dom.minidom.parse("D:/Paper S2/Data/Semval-2016/EN_REST_SB1_TEST.xml.gold")
topic=dom.getElementsByTagName('sentence')

xml_text, xml_target, xml_aspect, xml_sentiment = ReadXml(dom,topic)

xml_text, xml_target, xml_aspect, xml_sentiment = clear_double_target(xml_text, xml_target, xml_aspect, xml_sentiment)

# jml_data_train = jml_data_train_persentase(xml_text,0.1)
# xml_text_train, xml_text_test, xml_target_train, xml_target_test, xml_aspect_train, xml_aspect_test, xml_sentiment_train, xml_sentiment_test = split_to_test_and_train(xml_text, xml_target, xml_aspect, xml_sentiment,jml_data_train)
# print((xml_text_train))
# print("")
# print((xml_text_test))
print("===== Satu text dengan multi aspect =====")
print_data(xml_text, xml_target, xml_aspect, xml_sentiment,5)
# proses Preprocessing

arr = [] #hasil Spliting text sehingga tidak ada lagi satu text beberapa sentiment dan aspect
def xml_text_per_target(xml_text, xml_target, xml_aspect, xml_sentiment): 
    new_xml_text = []
    new_xml_target = []
    new_xml_aspect = []
    new_xml_sentiment = []
    for index_text in range(len(xml_text)):
    # for index_text in range(500): # Proses Split data dan preprocessing
    #     print(str(index_text)+". "+str(xml_text[index_text]))
        status_null= 0
        split_text_deleted_index = []
        if(len(xml_target[index_text])>1): # deteksi target per split text (not null target) 
            split_text = split_delimiter2(xml_text[index_text]) #Spliting data dengan delimiter
            i = 0
            while i < len(xml_target[index_text]):
                current_target = xml_target[index_text][i]
                if current_target == 'NULL':
                    status_null = 1
                j = 0
                while j < len(split_text):
                    if current_target in split_text[j] and " " in split_text[j]:
                        split_text_deleted_index.append(j)
                        new_xml_text.append(split_text[j])
                        new_xml_target.append(xml_target[index_text][i])
                        new_xml_aspect.append(xml_aspect[index_text][i])
                        new_xml_sentiment.append(xml_sentiment[index_text][i])
                        split_text[j] = ""
                    j+= 1
                i +=1
        if(status_null==1):
            index_null_target = xml_target[index_text].index('NULL')
            jj=0
            temp_len_split_text= -1
            index_split_text = -1
            while jj < len(split_text):
#            
                current_len_split_text = len(split_text[jj])
                if temp_len_split_text < current_len_split_text and current_len_split_text!='':
                    temp_len_split_text = current_len_split_text
                    index_split_text = jj
                jj+=1
            if index_split_text != -1 and " " in split_text[index_split_text]:
                new_xml_text.append(split_text[index_split_text])
                new_xml_target.append(xml_target[index_text][index_null_target])
                new_xml_aspect.append(xml_aspect[index_text][index_null_target])
                new_xml_sentiment.append(xml_sentiment[index_text][index_null_target])
        elif (str(xml_target[index_text]) !='[]' and len(xml_target[index_text])==1) :
            new_xml_text.append(xml_text[index_text])
            new_xml_target.append(xml_target[index_text][0])
            new_xml_aspect.append(xml_aspect[index_text][0])
            new_xml_sentiment.append(xml_sentiment[index_text][0])
#             print(xml_target[index_text][0])
    return new_xml_text, new_xml_target, new_xml_aspect, new_xml_sentiment
# Result After Preprocess
xml_text, xml_target,xml_aspect,xml_sentiment = xml_text_per_target(xml_text, xml_target, xml_aspect, xml_sentiment)
print("----> Menjadi <-----")
print("===== Satu text dengan satu aspect =====")
print_data_split_text(xml_text, xml_target,xml_aspect,xml_sentiment,5)
print(len(xml_text))
# print(arr)

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/alishalistya/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/alishalistya/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /Users/alishalistya/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /Users/alishalistya/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /Users/alishalistya/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_

===== Satu text dengan multi aspect =====
Text: Judging from previous posts this used to be a good place, but not any longer.
- Target: place | Aspect: RESTAURANT#GENERAL | Sentiment: negative

Text: We, there were four of us, arrived at noon - the place was empty - and the staff acted like we were imposing on them and they were very rude.
- Target: staff | Aspect: SERVICE#GENERAL | Sentiment: negative

Text: They never brought us complimentary noodles, ignored repeated requests for sugar, and threw our dishes on the table.
- Target: NULL | Aspect: SERVICE#GENERAL | Sentiment: negative

Text: The food was lousy - too sweet or too salty and the portions tiny.
- Target: food | Aspect: FOOD#QUALITY | Sentiment: negative
- Target: portions | Aspect: FOOD#STYLE_OPTIONS | Sentiment: negative

Text: After all that, they complained to me about the small tip.
- Target: NULL | Aspect: SERVICE#GENERAL | Sentiment: negative

----> Menjadi <-----
===== Satu text dengan satu aspect =====
Text: Judgi

Delete Neutral

In [152]:
# print_data_split_text(xml_text, xml_target,xml_aspect,xml_sentiment,5)
print(len(xml_text))
def delete_neutral(xml_text, xml_target,xml_aspect,xml_sentiment):
  new_xml_text = []
  new_xml_target = []
  new_xml_aspect = []
  new_xml_sentiment = []
  for jj in range(len(xml_text)):
    if xml_sentiment[jj]!="neutral":
      new_xml_text.append(xml_text[jj])
      new_xml_target.append(xml_target[jj])
      new_xml_aspect.append(xml_aspect[jj])
      new_xml_sentiment.append(xml_sentiment[jj])
  return new_xml_text, new_xml_target, new_xml_aspect, new_xml_sentiment
xml_text, xml_target,xml_aspect,xml_sentiment= delete_neutral(xml_text, xml_target,xml_aspect,xml_sentiment)
print(len(xml_text))

2153
2058


Read OWL

In [183]:
from owlready2 import *
onto = get_ontology("ontology.owl").load()
list(onto.properties())

mention = []
mention_aspect = []
mention_sentiment = []
mention_subclass = []
mention_axiom = []
mention_lex = []

def save_mention(name_mention,name_aspect,name_sentiment,name_subclass,name_lex):
    
    if len(name_lex)==1 and name_mention not in mention:
        mention.append(name_mention)
        mention_aspect.append(name_aspect)
        mention_sentiment.append(name_sentiment)
        mention_subclass.append(name_subclass)
        mention_lex.append(name_lex)
    else:
        for i in name_lex:
            if i not in mention:
                mention.append(i)
                mention_aspect.append(name_aspect)
                mention_sentiment.append(name_sentiment)
                mention_subclass.append(name_subclass)
                mention_lex.append(name_lex)
    
for i in onto.Mention.subclasses():
    for j in i.subclasses():
        sentiment_current = ''
        aspect_current = []
        aspect_current = j.aspect
        subclass_current = j.name
        if str(j).find('Positive')!= -1:
            sentiment_current = 'positive'
        elif str(j).find('Negative')!= -1:
            sentiment_current = 'negative'
        elif str(j).find('Mention')== -1 and str(j).find('Positive')== -1 and str(j).find('Negative')== -1:
            save_mention(j.name.lower(),j.aspect,'',subclass_current,j.lex)
###### Tambahan
        if str(j).find('Ambience')!= -1:         
            aspect_current = ['AMBIENCE#GENERAL']
        elif str(j).find('Price')!= -1:
            aspect_current = ['DRINKS#PRICES', 'FOOD#PRICES']
        elif str(j).find('Service')!= -1:
            aspect_current = ['SERVICE#GENERAL']
###### Tmbahan            

        for k in j.subclasses():
            sentiment_current_lv2 = ''
            aspect_current_lv2 = []
            aspect_current_lv2 = k.aspect
            if str(k).find('Positive')!= -1:
                sentiment_current_lv2 = 'positive'
            elif str(k).find('Negative')!= -1:
                sentiment_current_lv2 = 'negative'
            if str(k).find('Mention')== -1 and str(k).find('Positive')== -1 and str(k).find('Negative')== -1:
                save_mention(k.name.lower(),aspect_current,sentiment_current,subclass_current,k.lex)
            for l in k.subclasses():
                if sentiment_current_lv2!='' and aspect_current_lv2:
                    save_mention(l.name.lower(),aspect_current_lv2,sentiment_current_lv2,subclass_current,l.lex)
                elif sentiment_current_lv2!='':
                    save_mention(l.name.lower(),aspect_current,sentiment_current_lv2,subclass_current,l.lex)
                elif aspect_current_lv2:
                    save_mention(l.name.lower(),aspect_current_lv2,sentiment_current,subclass_current,l.lex)
                else:
                    save_mention(l.name.lower(),aspect_current,sentiment_current,subclass_current,l.lex)
                for m in l.subclasses():
                    if sentiment_current_lv2!='' and aspect_current_lv2:
                        save_mention(m.name.lower(),aspect_current_lv2,sentiment_current_lv2,subclass_current,m.lex)
                    elif sentiment_current_lv2!='':
                        save_mention(m.name.lower(),aspect_current,sentiment_current_lv2,subclass_current,m.lex)
                    elif aspect_current_lv2:
                        save_mention(m.name.lower(),aspect_current_lv2,sentiment_current,subclass_current,m.lex)
                    else:
                        save_mention(m.name.lower(),aspect_current,sentiment_current,subclass_current,m.lex)
                    for n in m.subclasses():
                        if sentiment_current_lv2!='' and aspect_current_lv2:
                            save_mention(n.name.lower(),aspect_current_lv2,sentiment_current_lv2,subclass_current,n.lex)
                        elif sentiment_current_lv2!='':
                            save_mention(n.name.lower(),aspect_current,sentiment_current_lv2,subclass_current,n.lex)
                        elif aspect_current_lv2:
                            save_mention(n.name.lower(),aspect_current_lv2,sentiment_current,subclass_current,n.lex)
                        else:
                            save_mention(n.name.lower(),aspect_current,sentiment_current,subclass_current,n.lex)

sentiment_term = []
sentiment = []
sentiment_aspect = []
for i in onto.Sentiment.subclasses():
#     print('*'+str(i))
    sentiment_current = ''
    sentiment_current = str(i.name.lower())
    for j in i.subclasses():
        aspect_current = []
        ###### Tambahan
        if str(j).find('Ambience')!= -1:         
            aspect_current = ['AMBIENCE#GENERAL']
        elif str(j).find('Price')!= -1:
            aspect_current = ['DRINKS#PRICES', 'FOOD#PRICES']
        elif str(j).find('Service')!= -1:
            aspect_current = ['SERVICE#GENERAL']
        ###### Tmbahan
        for k in j.subclasses():
            sentiment_term.append(k.name.lower())
            sentiment.append(sentiment_current)
            sentiment_aspect.append(aspect_current)

mention.append("beautiful")
mention_aspect.append([])
mention_sentiment.append("positive")
mention_subclass.append("")
mention_lex.append(["beautiful"])

print("{:<14} {:<13} {:<15} {:<10}".format("Term", "Sentiment","Aspect", "Sub_Class"))
print("---------------------------------------------------------------")

for i in range(len(mention)):
# for i in range(10):
    # if len(mention_aspect[i])==1:
    #   print ("{:<14} {:<13} {:<15} {:<10} {:<15}".format(str(mention[i]), str(mention_sentiment[i]),str(mention_aspect[i]), str(mention_subclass[i]),str(mention_lex[i])))
    if mention[i] =='expensive':
      print ("{:<14} {:<13} {:<15} {:<10} {:<15}".format(str(mention[i]), str(mention_sentiment[i]),str(mention_aspect[i]), str(mention_subclass[i]),str(mention_lex[i])))
print("")  
# print ("{:<14} {:<13} {:<15} ".format("Term", "Sentiment","Aspect"))
# print("---------------------------------------------------------------")
# for i in range(len(sentiment_term)):
#     print ("{:<14} {:<13} {:<15} ".format(str(sentiment_term[i]), str(sentiment[i]),str(sentiment_aspect[i])))

Term           Sentiment     Aspect          Sub_Class 
---------------------------------------------------------------
expensive      negative      ['DRINKS#PRICES', 'FOOD#PRICES'] PriceMention ['expensive']  



Sinonim Wordnet

In [154]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/alishalistya/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/alishalistya/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [155]:
from nltk.corpus import wordnet
def sinonim(word):
    synonyms = []
    for syn in wordnet.synsets(word):
        for lm in syn.lemmas():
                 synonyms.append(lm.name())#adding into synonyms
    return(set(synonyms))
sinonim("enjoy")
sinonim("terrace")

{'bench', 'patio', 'terrace', 'terrasse'}

In [156]:
def antonim(word):
    antonyms = []
    for syn in wordnet.synsets(word):
        for lm in syn.lemmas():
            if lm.antonyms():
                antonyms.append(lm.antonyms()[0].name()) #adding into antonyms
    return(set(antonyms))
antonim("increase")

{'decrease', 'decrement'}

fitur sentiment dan aspect

In [157]:
fiture_sentiment = []
fiture_aspect = []
for i in range(len(mention)):
    if mention[i] not in fiture_aspect:
      fiture_aspect.append(mention[i])
    if mention[i] not in fiture_sentiment:
      fiture_sentiment.append(mention[i])
    if mention_sentiment[i] != '':
        for sin in sinonim(mention[i]): # untuk memasukan ekstraksi fiture sinonim
            if sin not in fiture_sentiment:
              fiture_sentiment.append(sin)

    if len(mention_aspect[i])!=0:
        for sin in sinonim(mention[i]): # untuk memasukan ekstraksi fiture sinonim
            if sin not in fiture_aspect:
              fiture_aspect.append(sin)

Jalankan Preprocessing

In [158]:
for x in range(len(xml_text)):
    xml_text[x] = preprocessing(xml_text[x])
    
for x in range(5):
    print(xml_text[x])
print("['the', 'food', 'was', 'well', 'prepared']")

['judging', 'from', 'previous', 'posts', 'this', 'used', 'to', 'be', 'a', 'good', 'place', ',', 'but', 'not', 'any', 'longer', '.']
['we', ',', 'there', 'were', 'four', 'of', 'us', ',', 'arrived', 'at', 'noon', '-', 'the', 'place', 'was', 'empty', '-', 'and', 'the', 'staff', 'acted', 'like', 'we', 'were', 'imposing', 'on', 'them', 'and', 'they', 'were', 'very', 'rude', '.']
['they', 'never', 'brought', 'us', 'complimentary', 'noodles', ',', 'ignored', 'repeated', 'requests', 'for', 'sugar', ',', 'and', 'threw', 'our', 'dishes', 'on', 'the', 'table', '.']
['the', 'food', 'was', 'lousy', '-', 'too', 'sweet', 'or', 'too', 'salty']
['the', 'portions', 'tiny', '.']
['the', 'food', 'was', 'well', 'prepared']


Split Data Text

In [159]:

from sklearn import svm, datasets
import sklearn.model_selection as model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

temp_arr = []


X = xml_text
y = xml_sentiment
y_aspect = xml_aspect

text_train, text_test, sentiment_train, sentiment_test = model_selection.train_test_split(X, y, train_size=0.70, test_size=0.30, random_state=101)
text_train, text_test, aspect_train, aspect_test = model_selection.train_test_split(X, y_aspect, train_size=0.70, test_size=0.30, random_state=101)

Klasifikasi Ontology Sentiment

In [160]:
print(len(text_test))
from strsimpy.qgram import QGram

qgram = QGram(2)
print(qgram.distance('Doni-aaaa', 'Doni-aaaa'))

me = MongeElkan()
arr_id_klasi_onto = []
arr_senti_klasi_onto = []

arr_id_aspect_klasi_onto = []
arr_aspect_klasi_onto = []

xx = 0
for z in range(len(text_test)):
  temp = 10
  temp2 = 10
  conflict = 0 # 0 no conflict - 1 conflict
  jml_match = 0
  jml_match_aspect = 0
  conflict_aspect = 0 # 0 no conflict - 1 conflict
  for zz in range(len(text_test[z])):
    for i in range(len(mention)):
       if (mention_sentiment[i] != ''):
        score = qgram.distance(str([mention[i]]), str([text_test[z][zz]]))
        if score < 1 :
          jml_match = jml_match + 1
          if jml_match == 1:
            hasil_sentiment_klasi_onto = mention_sentiment[i]
            hasil_id_klasi_onto = z
          elif hasil_sentiment_klasi_onto != mention_sentiment[i]:
            conflict = 1
    for i in range(len(mention)):
       if len(mention_aspect[i])==1:
        score2 = qgram.distance(str([mention[i]]), str([text_test[z][zz]]))
        if score2 < 1 :
          jml_match_aspect = jml_match_aspect + 1
          if jml_match_aspect == 1:
            hasil_aspect_klasi_onto = mention_aspect[i]
            hasil_id_aspect_klasi_onto = z
          elif hasil_aspect_klasi_onto != mention_aspect[i]:
            conflict_aspect = 1
  if jml_match > 0 and conflict != 1:
    arr_id_klasi_onto.append(hasil_id_klasi_onto)
    arr_senti_klasi_onto.append(hasil_sentiment_klasi_onto)
  if jml_match_aspect > 0 and conflict_aspect != 1:
    arr_id_aspect_klasi_onto.append(hasil_id_aspect_klasi_onto)
    arr_aspect_klasi_onto.append(hasil_aspect_klasi_onto)
  xx = xx + 1

print(len(arr_id_klasi_onto))
print(arr_id_klasi_onto)
print(arr_senti_klasi_onto)

print(arr_id_aspect_klasi_onto)
print(arr_aspect_klasi_onto)


618
0.0
307
[1, 2, 3, 7, 8, 10, 11, 14, 16, 28, 30, 32, 34, 39, 40, 41, 42, 44, 48, 49, 50, 51, 53, 54, 56, 57, 61, 62, 65, 67, 70, 71, 77, 78, 79, 84, 86, 90, 91, 93, 94, 95, 99, 100, 102, 103, 104, 105, 106, 107, 108, 109, 110, 113, 115, 117, 120, 122, 123, 124, 125, 126, 130, 131, 132, 134, 138, 139, 140, 141, 142, 143, 144, 145, 147, 153, 154, 156, 160, 161, 163, 167, 168, 169, 171, 172, 173, 174, 175, 176, 177, 179, 181, 183, 186, 191, 192, 193, 201, 202, 203, 204, 207, 209, 210, 211, 212, 213, 215, 217, 219, 220, 222, 230, 231, 232, 233, 235, 236, 237, 238, 240, 241, 244, 247, 249, 253, 255, 256, 257, 258, 260, 262, 264, 265, 267, 268, 270, 273, 274, 276, 277, 278, 279, 281, 283, 284, 287, 288, 290, 292, 293, 294, 295, 299, 300, 302, 304, 305, 307, 314, 319, 321, 324, 325, 328, 331, 332, 333, 338, 339, 345, 347, 348, 349, 352, 355, 357, 360, 361, 362, 363, 364, 368, 372, 373, 374, 375, 376, 381, 387, 389, 390, 391, 396, 397, 398, 400, 402, 403, 404, 408, 411, 414, 416, 420, 421, 

Hasil Sentiment

In [262]:
question_1 = "you sure get a lot of food for your money."
question_1  = preprocessing(question_1 )

for word in range(len(question_1 )):
  for i in range(len(mention)):
       if (mention_sentiment[i] != ''):
        score = qgram.distance(str([mention[i]]), str([question_1 [word]]))
        if score < 1 :
          # print(qgram.distance(str([mention[i]]), str([text_test[z][zz]])))
          # print(text_test[z][zz]+str(mention[i])+" "+str(text_test[z][zz]))
          jml_match = jml_match + 1
          if jml_match == 1:
            # temp = score
            hasil_sentiment_klasi_onto = mention_sentiment[i]
            hasil_id_klasi_onto = z
            # print("1-"+str([mention[i]]), str([text_test[z][zz]]))
          elif hasil_sentiment_klasi_onto != mention_sentiment[i]:
            hasil_sentiment_klasi_onto = mention_sentiment[i]
            hasil_id_klasi_onto = z
            # print("2-"+str([mention[i]]), str([text_test[z][zz]]))
            # print(hasil_sentiment_klasi_onto + mention_sentiment[i])
            conflict = 1
# print("3-"+str([mention[i]]), str([text_test[z][zz]]))
# print(hasil_sentiment_klasi_onto + mention_sentiment[i])
print(str(hasil_sentiment_klasi_onto)+str(hasil_id_klasi_onto))

3-['beautiful'] ['fish']
negativepositive
negative617
